#Name of student: ADITI PALA


loading the datasets

In [ ]:
import pandas as pd

# Import necessary libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string

In [ ]:
from google.colab import drive
drive.mount("/drive", force_remount=True)
import csv
import pandas as pd
true_data = pd.read_csv('/drive/My Drive/Colab Notebooks/IST 664/2023/True.csv')


Mounted at /drive


In [ ]:
fake_data=pd.read_csv('/drive/My Drive/Colab Notebooks/IST 664/2023/Fake.csv')

In [ ]:
display(true_data.head())

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
display(fake_data.head())

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


##*Data Preprocessing*

---



In [ ]:
print(len(true_data))
print(len(fake_data))

21417
23481


In [ ]:
missing_rows_t = true_data[true_data['text'].isna()]

# Check if there are any missing rows
if not missing_rows_t.empty:
    print(f"There are {len(missing_rows_t)} missing rows in the true data set text column.")
else:
    print("There are no missing values in the true data set text column.")


There are no missing values in the true data set text column.


In [ ]:
missing_rows_f = fake_data[fake_data['text'].isna()]

# Check if there are any missing rows
if not missing_rows_f.empty:
    print(f"There are {len(missing_rows_f)} missing rows in the fake data set text column.")
else:
    print("There are no missing values in the fake data set text column.")


There are no missing values in the fake data set text column.


Tokenize

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Why Tokenization?

Tokenizationis the process of splitting a text or a document into individual words or tokens.

In this data, we need to break text into smaller words in order to perform text analysis

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

true_data['text_tokenized'] = true_data['text'].apply(lambda x: word_tokenize(x))


In [ ]:
display(true_data.head())

,title,text,subject,date,text_tokenized
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017","[WASHINGTON, (, Reuters, ), -, The, head, of, ..."
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017","[WASHINGTON, (, Reuters, ), -, Transgender, pe..."
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017","[WASHINGTON, (, Reuters, ), -, The, special, c..."
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017","[WASHINGTON, (, Reuters, ), -, Trump, campaign..."
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017","[SEATTLE/WASHINGTON, (, Reuters, ), -, Preside..."


In [ ]:

fake_data['text_tokenized'] = fake_data['text'].apply(lambda x: word_tokenize(x))


In [ ]:
display(fake_data.head())

,title,text,subject,date,text_tokenized
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017","[Donald, Trump, just, couldn, t, wish, all, Am..."
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017","[House, Intelligence, Committee, Chairman, Dev..."
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017","[On, Friday, ,, it, was, revealed, that, forme..."
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017","[On, Christmas, day, ,, Donald, Trump, announc..."
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017","[Pope, Francis, used, his, annual, Christmas, ..."


Creating copies of the original DataFrames to store results

so I can perform calculations and save the information in these DataFrames to obtain the required statistics for each article

In [ ]:
results_data_true=true_data.copy()
results_data_fake=fake_data.copy()

Why Converting tokens into lowercase?

This ensures uniformity in text analysis.

It helps treat words in different cases (e.g., "Trump" and "trump") as the same, preventing duplication of information and improving the accuracy of text-based operations.






In [ ]:
# Make tokens lowercase
true_data['text_tokenized_lower'] = true_data['text_tokenized'].apply(lambda x: [word.lower() for word in x])


In [ ]:
# Make tokens lowercase
fake_data['text_tokenized_lower'] = fake_data['text_tokenized'].apply(lambda x: [word.lower() for word in x])


In [ ]:
display(true_data.head())

,title,text,subject,date,text_tokenized,text_tokenized_lower
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017","[WASHINGTON, (, Reuters, ), -, The, head, of, ...","[washington, (, reuters, ), -, the, head, of, ..."
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017","[WASHINGTON, (, Reuters, ), -, Transgender, pe...","[washington, (, reuters, ), -, transgender, pe..."
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017","[WASHINGTON, (, Reuters, ), -, The, special, c...","[washington, (, reuters, ), -, the, special, c..."
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017","[WASHINGTON, (, Reuters, ), -, Trump, campaign...","[washington, (, reuters, ), -, trump, campaign..."
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017","[SEATTLE/WASHINGTON, (, Reuters, ), -, Preside...","[seattle/washington, (, reuters, ), -, preside..."


In [ ]:
display(fake_data.head())

,title,text,subject,date,text_tokenized,text_tokenized_lower
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017","[Donald, Trump, just, couldn, t, wish, all, Am...","[donald, trump, just, couldn, t, wish, all, am..."
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017","[House, Intelligence, Committee, Chairman, Dev...","[house, intelligence, committee, chairman, dev..."
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017","[On, Friday, ,, it, was, revealed, that, forme...","[on, friday, ,, it, was, revealed, that, forme..."
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017","[On, Christmas, day, ,, Donald, Trump, announc...","[on, christmas, day, ,, donald, trump, announc..."
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017","[Pope, Francis, used, his, annual, Christmas, ...","[pope, francis, used, his, annual, christmas, ..."


Why Remove Non-words?

The text data when checking for fake news becomes noisy when non-words like punctuation, special characters, and digits are used. By getting rid of them, we can lessen the background noise and concentrate on the text's important ideas.

In [ ]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
print(type(true_data['text_tokenized_lower'].iloc[0]))


<class 'list'>


In [ ]:
import string

# Define a function to remove non-words and return tokens
def remove_non_words(tokens):
    # Filter out tokens that contain digits or special characters
    cleaned_tokens = [token for token in tokens if all(char.isalpha() or char.isspace() for char in token)]

    return cleaned_tokens


In [ ]:
true_data['text_words'] = true_data['text_tokenized_lower'].apply(remove_non_words)

In [ ]:
fake_data['text_words'] = fake_data['text_tokenized_lower'].apply(remove_non_words)

In [ ]:
display(true_data.head())

,title,text,subject,date,text_tokenized,text_tokenized_lower,text_words
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017","[WASHINGTON, (, Reuters, ), -, The, head, of, ...","[washington, (, reuters, ), -, the, head, of, ...","[washington, reuters, the, head, of, a, conser..."
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017","[WASHINGTON, (, Reuters, ), -, Transgender, pe...","[washington, (, reuters, ), -, transgender, pe...","[washington, reuters, transgender, people, wil..."
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017","[WASHINGTON, (, Reuters, ), -, The, special, c...","[washington, (, reuters, ), -, the, special, c...","[washington, reuters, the, special, counsel, i..."
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017","[WASHINGTON, (, Reuters, ), -, Trump, campaign...","[washington, (, reuters, ), -, trump, campaign...","[washington, reuters, trump, campaign, adviser..."
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017","[SEATTLE/WASHINGTON, (, Reuters, ), -, Preside...","[seattle/washington, (, reuters, ), -, preside...","[reuters, president, donald, trump, called, on..."


In [ ]:
display(fake_data.head())

,title,text,subject,date,text_tokenized,text_tokenized_lower,text_words
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017","[Donald, Trump, just, couldn, t, wish, all, Am...","[donald, trump, just, couldn, t, wish, all, am...","[donald, trump, just, couldn, t, wish, all, am..."
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017","[House, Intelligence, Committee, Chairman, Dev...","[house, intelligence, committee, chairman, dev...","[house, intelligence, committee, chairman, dev..."
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017","[On, Friday, ,, it, was, revealed, that, forme...","[on, friday, ,, it, was, revealed, that, forme...","[on, friday, it, was, revealed, that, former, ..."
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017","[On, Christmas, day, ,, Donald, Trump, announc...","[on, christmas, day, ,, donald, trump, announc...","[on, christmas, day, donald, trump, announced,..."
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017","[Pope, Francis, used, his, annual, Christmas, ...","[pope, francis, used, his, annual, christmas, ...","[pope, francis, used, his, annual, christmas, ..."


Why are we doing Lemmatization?

Lemmatization helps in reducing these different forms to their base or root form.

Lemmatization simplifies word forms for consistent text analysis, aiding in distinguishing fake from real news by capturing nuanced word usage and context.

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

In [ ]:
# Apply the function to the 'text_tokenized' column
true_data['text_lemma'] = true_data['text_words'].apply(lemmatize_tokens)

In [ ]:
# Apply the function to the 'text_tokenized' column
fake_data['text_lemma'] = fake_data['text_words'].apply(lemmatize_tokens)

In [ ]:
display(true_data.head())

,title,text,subject,date,text_tokenized,text_tokenized_lower,text_words,text_lemma
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017","[WASHINGTON, (, Reuters, ), -, The, head, of, ...","[washington, (, reuters, ), -, the, head, of, ...","[washington, reuters, the, head, of, a, conser...","[washington, reuters, the, head, of, a, conser..."
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017","[WASHINGTON, (, Reuters, ), -, Transgender, pe...","[washington, (, reuters, ), -, transgender, pe...","[washington, reuters, transgender, people, wil...","[washington, reuters, transgender, people, wil..."
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017","[WASHINGTON, (, Reuters, ), -, The, special, c...","[washington, (, reuters, ), -, the, special, c...","[washington, reuters, the, special, counsel, i...","[washington, reuters, the, special, counsel, i..."
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017","[WASHINGTON, (, Reuters, ), -, Trump, campaign...","[washington, (, reuters, ), -, trump, campaign...","[washington, reuters, trump, campaign, adviser...","[washington, reuters, trump, campaign, adviser..."
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017","[SEATTLE/WASHINGTON, (, Reuters, ), -, Preside...","[seattle/washington, (, reuters, ), -, preside...","[reuters, president, donald, trump, called, on...","[reuters, president, donald, trump, called, on..."


In [ ]:
display(fake_data.head())

,title,text,subject,date,text_tokenized,text_tokenized_lower,text_words,text_lemma
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017","[Donald, Trump, just, couldn, t, wish, all, Am...","[donald, trump, just, couldn, t, wish, all, am...","[donald, trump, just, couldn, t, wish, all, am...","[donald, trump, just, couldn, t, wish, all, am..."
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017","[House, Intelligence, Committee, Chairman, Dev...","[house, intelligence, committee, chairman, dev...","[house, intelligence, committee, chairman, dev...","[house, intelligence, committee, chairman, dev..."
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017","[On, Friday, ,, it, was, revealed, that, forme...","[on, friday, ,, it, was, revealed, that, forme...","[on, friday, it, was, revealed, that, former, ...","[on, friday, it, wa, revealed, that, former, m..."
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017","[On, Christmas, day, ,, Donald, Trump, announc...","[on, christmas, day, ,, donald, trump, announc...","[on, christmas, day, donald, trump, announced,...","[on, christmas, day, donald, trump, announced,..."
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017","[Pope, Francis, used, his, annual, Christmas, ...","[pope, francis, used, his, annual, christmas, ...","[pope, francis, used, his, annual, christmas, ...","[pope, francis, used, his, annual, christmas, ..."


Checking for rows with missing/NaN values after data pre-processing is completed


In [ ]:
missing_rows_after_processing_t = true_data[true_data['text_lemma'].isna()]

# Check if there are any missing rows
if not missing_rows_after_processing_t.empty:
    print(f"There are {len(missing_rows_after_processing_t)} missing rows in the true data set text column.")
else:
    print("There are no missing values in the true data set text column.")


There are no missing values in the true data set text column.


In [ ]:
missing_rows_after_processing_f = fake_data[fake_data['text_lemma'].isna()]

# Check if there are any missing rows
if not missing_rows_after_processing_f.empty:
    print(f"There are {len(missing_rows_after_processing_f)} missing rows in the fake data set text column.")
else:
    print("There are no missing values in the fake data set text column.")


There are no missing values in the fake data set text column.


#Data Analysis

##*1. list the top 50 stop words by frequency in fake news articles and those in real news articles*

---



For true dataset

In [ ]:
import pandas as pd
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Flatten the list of tokens into a single list for real data
true_tokens = [token for tokens_list in true_data['text_lemma'] for token in tokens_list]

In [ ]:
# Count the frequency of each word for real data
true_word_freq = Counter(true_tokens)

In [ ]:
top_50_true_stopwords = [(word, freq) for word, freq in true_word_freq.items() if word in stop_words]
top_50_true_stopwords.sort(key=lambda x: x[1], reverse=True)

In [ ]:
top_50_true_stopwords = top_50_true_stopwords[:50]


In [ ]:
# Create a DataFrame for the top 50 stop words in real news articles
top_50_true_df = pd.DataFrame(top_50_true_stopwords, columns=['Word', 'Frequency'])

# Print the DataFrame
print("Top 50 Stop Words in Real News Articles:")
print(top_50_true_df)

Top 50 Stop Words in Real News Articles:
       Word  Frequency
0       the     482176
1         a     245167
2        to     245125
3        of     204959
4       and     181686
5        in     180675
6        on     108346
7         s      99905
8      that      88116
9       for      79730
10       it      70683
11       is      55439
12     with      54485
13       he      54368
14       by      47637
15     from      39376
16      his      38016
17      not      37594
18     have      36415
19       be      34297
20       at      33164
21       an      32710
22      who      27642
23      but      26596
24      are      26127
25      had      25672
26     will      25257
27       we      22163
28     they      22064
29     this      21025
30    which      20809
31     been      19600
32    after      19375
33     were      18916
34    about      17425
35     more      17200
36        i      17020
37    their      16826
38       or      15107
39     over      13841
40     some     

For Fake data


Fake data has missing rows or NaN data

In [ ]:
# Filter out rows where 'text_lemma' is not NaN
fake_data = fake_data[fake_data['text_lemma'].notna()]

In [ ]:
fake_tokens = [token for tokens_list in fake_data['text_lemma'] for token in tokens_list]


In [ ]:
# Count the frequency of each word
fake_word_freq = Counter(fake_tokens)

In [ ]:
top_50_fake_stopwords = [(word, freq) for word, freq in fake_word_freq.items() if word in stop_words]
top_50_fake_stopwords.sort(key=lambda x: x[1], reverse=True)

In [ ]:
top_50_fake_stopwords = top_50_fake_stopwords[:50]


In [ ]:
# Create a DataFrame for the top 50 stop words in fake news articles
top_50_fake_df = pd.DataFrame(top_50_fake_stopwords, columns=['Word', 'Frequency'])

# Print the DataFrame
print("Top 50 Stop Words in Fake News Articles:")
print(top_50_fake_df)

Top 50 Stop Words in Fake News Articles:
     Word  Frequency
0     the     528689
1      to     289665
2       a     267070
3      of     235806
4     and     224544
5      in     164871
6    that     150262
7       s     129672
8      is     110887
9     for      92115
10     it      89002
11     on      81354
12     he      77635
13   with      62964
14    his      58037
15   this      55655
16     be      48629
17     by      47387
18    not      46833
19    are      46350
20   have      46043
21   they      45190
22      i      43778
23    who      42327
24     at      41569
25   from      40754
26      t      40541
27    you      40323
28     we      38205
29     an      34864
30  about      32834
31    but      31426
32  their      30202
33    she      25763
34    her      25705
35     or      24671
36   what      24594
37    all      24455
38   will      24069
39   been      22991
40    out      22936
41   more      22395
42     if      22221
43   were      21608
44   when     

##*2. list the top 50 content words by frequency in fake news articles and those in real news articles*

---



To find content words, we need to filter out stop words

Finding content words from True Dataset

In [ ]:
# Remove stop words
true_content_words = [word for word in true_tokens if word not in stop_words]


In [ ]:
# Count the frequency of each content word in real data
true_content_word_freq = Counter(true_content_words)


In [ ]:
# Get the top 50 content words by frequency for real news articles
top_50_true_content_words = true_content_word_freq.most_common(50)

In [ ]:
# Create a DataFrame for the top 50 content words in real news articles
top_50_true_content_df = pd.DataFrame(top_50_true_content_words, columns=['Word', 'Frequency'])

In [ ]:
# Print the DataFrame
print("Top 50 Content Words in Real News Articles:")
print(top_50_true_content_df)


Top 50 Content Words in Real News Articles:
          Word  Frequency
0         said      99037
1        trump      54280
2           wa      48027
3           ha      46259
4        state      36243
5        would      31526
6      reuters      28412
7    president      26939
8   republican      22098
9   government      19433
10        year      18721
11       house      16906
12         new      16784
13        also      15946
14      united      15576
15      people      15207
16       party      14964
17    official      14575
18        told      14244
19     country      13948
20    election      13910
21       could      13709
22         one      13025
23        last      12631
24  washington      12419
25         two      11620
26       group      11104
27    campaign      11079
28      former      10601
29      leader      10504
30      donald      10447
31        week      10426
32    security      10382
33       court      10338
34     percent       9937
35         say      

finding content words from fake data

In [ ]:
# Remove stop words
fake_content_words = [word for word in fake_tokens if word not in stop_words]


In [ ]:
# Count the frequency of each content word in fake data
fake_content_word_freq = Counter(fake_content_words)


In [ ]:
# Get the top 50 content words by frequency for fake news articles
top_50_fake_content_words = fake_content_word_freq.most_common(50)


In [ ]:
# Create a DataFrame for the top 50 content words in fake news articles
top_50_fake_content_df = pd.DataFrame(top_50_fake_content_words, columns=['Word', 'Frequency'])

# Print the DataFrame
print("Top 50 Content Words in Fake News Articles:")
print(top_50_fake_content_df)

Top 50 Content Words in Fake News Articles:
          Word  Frequency
0        trump      74403
1           wa      67826
2           ha      42348
3         said      31149
4    president      26340
5       people      26098
6          one      23812
7        would      23461
8        state      22072
9      clinton      18717
10        like      18207
11       obama      17920
12        time      17885
13      donald      17235
14    american      16093
15  republican      16061
16         say      15528
17        also      15243
18        year      14843
19         new      14198
20        news      14198
21           u      14181
22       image      13937
23        even      13692
24     hillary      13678
25       white      13146
26       right      12698
27         get      12230
28        know      11947
29        make      11534
30         via      11355
31       woman      11207
32      medium      11142
33    campaign      11069
34       house      10774
35     country      

##*3. list the top 50 bigrams by frequencies in fake news articles and those in real news articles*

---



So the text_lemma column of both datasets have stopwords & content words both.

Hence, if we create bigrams directly, we may receive bigrams with combinations like:
1. *stopword with stopwords*
- for example: in the

2. *stopwords with content words*
- for example: the president

3. *content words with stopwords*
- for example: news on

4. *content words with content words*
- for example: breaking news

True dataset - check for bigrams

Hence, I am using the previously initialised true_content_words which is a list of words without stop words

In [ ]:
# Remove stop words
#for reference, im re-initialising this again
true_content_words = [word for word in true_tokens if word not in stop_words]


In [ ]:
def get_top_50_bigrams(tokens, n=50):
    bigrams = [(tokens[i], tokens[i+1]) for i in range(len(tokens) - 1)]
    bigram_freq = Counter(bigrams)
    top_bigrams = bigram_freq.most_common(n)
    return top_bigrams


In [ ]:
# Get the top 50 bigrams by frequency in true news articles using the custom function
top_50_true_bigrams = get_top_50_bigrams(true_content_words, n=50)


In [ ]:

# Create a DataFrame for the top 50 bigrams in true news articles
top_50_true_bigrams_df = pd.DataFrame(top_50_true_bigrams, columns=['Bigram', 'Frequency'])

# Print the DataFrame
print("Top 50 Bigrams in True News Articles:")
display(top_50_true_bigrams_df)

Top 50 Bigrams in True News Articles:


,Bigram,Frequency
0,"(united, state)",12096
1,"(donald, trump)",10147
2,"(white, house)",8372
3,"(washington, reuters)",6497
4,"(president, donald)",5925
5,"(north, korea)",5629
6,"(new, york)",4450
7,"(prime, minister)",4175
8,"(said, statement)",3954
9,"(said, wa)",3746


Fake dataset - check for bigrams

Hence, I am using the previously initialised fake_content_words which is a list of words without stop words

In [ ]:
# Remove stop words
#for reference, I'm re-initialising it
fake_content_words = [word for word in fake_tokens if word not in stop_words]


In [ ]:
# Get the top 50 bigrams by frequency in fake news articles using the custom function
top_50_fake_bigrams = get_top_50_bigrams(fake_content_words, n=50)


In [ ]:
# Create a DataFrame for the top 50 bigrams in fake news articles
top_50_fake_bigrams_df = pd.DataFrame(top_50_fake_bigrams, columns=['Bigram', 'Frequency'])

# Print the DataFrame
print("Top 50 Bigrams in Fake News Articles:")
display(top_50_fake_bigrams_df)

Top 50 Bigrams in Fake News Articles:


,Bigram,Frequency
0,"(donald, trump)",15361
1,"(hillary, clinton)",6788
2,"(white, house)",6349
3,"(image, via)",6232
4,"(united, state)",6209
5,"(new, york)",4209
6,"(president, obama)",3857
7,"(president, trump)",3767
8,"(trump, ha)",3502
9,"(fox, news)",3243


## *4. list the top 50 bigrams by their Mutual Information scores (using min frequency 5) in fake news articles and those in real news articles*

---



Mutual Information (MI) is a statistical metric used in fake news analysis to measure the strength of association between two words or tokens.

It helps identify meaningful word pairs that frequently appear together in fake news articles, revealing important patterns in fake news content.

In the last question(top 50 bigrams), we found out only top 50 bigrams

Here we want to find all bigrams, calulate their scores and then find top 50 occurences

True data

In [ ]:
all_bigrams_t = [(true_content_words[i], true_content_words[i+1]) for i in range(len(true_content_words) - 1)]

In [ ]:
true_content_word_freq = Counter(true_content_words)

In [ ]:
# Calculate bigram frequencies
bigram_freq_t = Counter(all_bigrams_t)

In [ ]:
# Calculate corpus size
size_content_words_t = len(true_content_words)

In [ ]:
mi_scores_true = []

In [ ]:
# Calculate MI score for each bigram
import math
for bigram in all_bigrams_t:
    word1, word2 = bigram
    bigram_count = bigram_freq_t[bigram]
    word1_count = true_content_word_freq[word1]
    word2_count = true_content_word_freq[word2]

    if bigram_count >= 5: #making min frequency as 5
      mi_score = math.log((size_content_words_t * bigram_count) / (word1_count * word2_count), 2)
      mi_scores_true.append((bigram, mi_score))

In [ ]:
# Sort bigrams by MI score in descending order for true content
mi_scores_true.sort(key=lambda x: x[1], reverse=True)

In [ ]:
top_50_bigrams_MI_true = mi_scores_true[:50]

In [ ]:
# Create a DataFrame for the top 50 bigrams with columns 'Bigram' and 'MI Score'
top_50_bigrams_MI_true_df = pd.DataFrame(top_50_bigrams_MI_true, columns=['Bigram', 'MI Score'])

# Print the DataFrame along with a descriptive message
print("Top 50 Bigrams by MI Scores in True News Articles:")
display(top_50_bigrams_MI_true_df)

Top 50 Bigrams by MI Scores in True News Articles:


,Bigram,MI Score
0,"(keefe, bruyette)",19.869401
1,"(keefe, bruyette)",19.869401
2,"(maale, adumim)",19.869401
3,"(maale, adumim)",19.869401
4,"(maale, adumim)",19.869401
5,"(doxycycline, hyclate)",19.869401
6,"(doxycycline, hyclate)",19.869401
7,"(doxycycline, hyclate)",19.869401
8,"(doxycycline, hyclate)",19.869401
9,"(doxycycline, hyclate)",19.869401


Fake news

In [ ]:
fake_content_word_freq = Counter(fake_content_words)

In [ ]:
all_bigrams_f = [(fake_content_words[i], fake_content_words[i+1]) for i in range(len(fake_content_words) - 1)]

In [ ]:
# Calculate all bigram frequencies
bigram_freq_f = Counter(all_bigrams_f)

In [ ]:
# Calculate corpus size
size_content_words_f = len(fake_content_words)

In [ ]:
mi_scores_fake = []

In [ ]:
# Calculate MI score for each unique bigram
for bigram, bigram_count in bigram_freq_f.items():
    word1, word2 = bigram
    word1_count = fake_content_word_freq[word1]
    word2_count = fake_content_word_freq[word2]

    if bigram_count >= 5:  # Minimum frequency as 5
        mi_score = math.log((size_content_words_f * bigram_count) / (word1_count * word2_count), 2)
        mi_scores_fake.append((bigram, mi_score))

In [ ]:

# Sort bigrams by MI score in descending order
mi_scores_fake.sort(key=lambda x: x[1], reverse=True)

top_50_bigrams_MI_fake = mi_scores_fake[:50]



In [ ]:
# Create a DataFrame for the top 50 bigrams with columns 'Bigram' and 'MI Score'
import pandas as pd

top_50_bigrams_MI_fake_df = pd.DataFrame(top_50_bigrams_MI_fake, columns=['Bigram', 'MI Score'])



In [ ]:
# Print the DataFrame along with a descriptive message
print("Top 50 Bigrams by MI Scores in Fake News Articles:")
display(top_50_bigrams_MI_fake_df)

Top 50 Bigrams by MI Scores in Fake News Articles:


,Bigram,MI Score
0,"(abravanel, lesleyabravanel)",20.013403
1,"(hemmed, hawed)",20.013403
2,"(boothby, ianboothby)",20.013403
3,"(svokos, asvokos)",20.013403
4,"(pragides, mrp)",20.013403
5,"(kapur, sahilkapur)",20.013403
6,"(palos, verdes)",20.013403
7,"(platformvideo, managementvideo)",20.013403
8,"(managementvideo, solutionsvideo)",20.013403
9,"(myocardial, infarction)",20.013403


## *5. list top 50 adjective words in fake news articles and those in real news articles*

---



In [ ]:
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
from nltk import pos_tag
# Perform part-of-speech tagging
true_tags = pos_tag(true_content_words)

In [ ]:
true_adjectives = [word for word, tag in true_tags if tag.startswith('JJ')]


In [ ]:
true_adjective_freq = Counter(true_adjectives)

In [ ]:
top_50_real_adjectives_df = pd.DataFrame(true_adjective_freq.most_common(50))

In [ ]:
print("\nTop 50 Adjectives in Real News Articles:")
display(top_50_real_adjectives_df)


Top 50 Adjectives in Real News Articles:


,0,1
0,republican,20305
1,new,16784
2,united,13734
3,last,12631
4,former,10601
5,trump,9797
6,white,9500
7,national,8534
8,north,8278
9,democratic,8235


fake news


In [ ]:
from nltk import pos_tag
# Perform part-of-speech tagging
fake_tags = pos_tag(fake_content_words)

In [ ]:
fake_adjectives = [word for word, tag in fake_tags if tag.startswith('JJ')]


In [ ]:
fake_adjective_freq = Counter(fake_adjectives)

In [ ]:
top_50_fake_adjectives_df = pd.DataFrame(fake_adjective_freq.most_common(50))

In [ ]:
print("\nTop 50 Adjectives in fake News Articles:")
display(top_50_fake_adjectives_df)


Top 50 Adjectives in fake News Articles:


,0,1
0,american,16070
1,republican,14909
2,new,14198
3,white,13146
4,hillary,12891
5,u,12260
6,trump,11553
7,wa,10108
8,many,9719
9,black,8047


5. *For each article, you will also obtain total number of words, total number of content words, total number
of words that are all capitalized (excluding “I”), total number of exclamation marks, and total number
of punctuation marks. You will save this information in CSV files by creating new columns and saving
the information in these columns*

---



In [ ]:
import pandas as pd
import re
import csv

True statistics

In [ ]:
results_data_true['Total_Words'] = true_data['text_tokenized'].apply(len)

In [ ]:
def count_content_words(tokens, stop_words):
    content_words = [token for token in tokens if token not in stop_words]
    return len(content_words)

results_data_true['Total_Content_Words'] = true_data['text_tokenized'].apply(count_content_words, args=(stop_words,))


In [ ]:
def count_capitalized_words(tokens):
    return sum(1 for token in tokens if token.isupper() and token != 'I')

results_data_true['Total_Capitalized_Words'] = true_data['text_tokenized'].apply(count_capitalized_words)

In [ ]:
def count_exclamation_marks(tokens):
    return sum(1 for token in tokens if '!' in token)
results_data_true['Total_Exclamation_Marks'] = true_data['text_tokenized'].apply(count_exclamation_marks)

In [ ]:
import string
def count_punctuation_marks(tokens):
  punctuation = string.punctuation
  return sum(1 for token in tokens if token in punctuation)

results_data_true['Total_Punctuation_Marks'] = true_data['text_tokenized'].apply(count_punctuation_marks)

In [ ]:
display(results_data_true.head())

,title,text,subject,date,text_tokenized,Total_Words,Total_Content_Words,Total_Capitalized_Words,Total_Exclamation_Marks,Total_Punctuation_Marks
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017","[WASHINGTON, (, Reuters, ), -, The, head, of, ...",885,602,10,0,94
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017","[WASHINGTON, (, Reuters, ), -, Transgender, pe...",713,479,8,0,56
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017","[WASHINGTON, (, Reuters, ), -, The, special, c...",523,335,6,0,41
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017","[WASHINGTON, (, Reuters, ), -, Trump, campaign...",437,291,5,0,42
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017","[SEATTLE/WASHINGTON, (, Reuters, ), -, Preside...",987,660,15,1,111


fake statistics

In [ ]:
# Calculate and store statistics for each article in the fake news dataset
results_data_fake['Total_Words'] = fake_data['text_tokenized'].apply(len)


In [ ]:
results_data_fake['Total_Content_Words'] = fake_data['text_tokenized'].apply(count_content_words, args=(stop_words,))


In [ ]:
results_data_fake['Total_Capitalized_Words'] = fake_data['text_tokenized'].apply(count_capitalized_words)

In [ ]:
results_data_fake['Total_Exclamation_Marks'] = fake_data['text_tokenized'].apply(count_exclamation_marks)




In [ ]:
results_data_fake['Total_Punctuation_Marks'] = fake_data['text_tokenized'].apply(count_punctuation_marks)


In [ ]:
display(results_data_fake.head())

,title,text,subject,date,text_tokenized,Total_Words,Total_Content_Words,Total_Capitalized_Words,Total_Exclamation_Marks,Total_Punctuation_Marks
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017","[Donald, Trump, just, couldn, t, wish, all, Am...",599,406,2,6,107
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017","[House, Intelligence, Committee, Chairman, Dev...",331,211,3,0,26
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017","[On, Friday, ,, it, was, revealed, that, forme...",689,469,33,2,109
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017","[On, Christmas, day, ,, Donald, Trump, announc...",519,356,4,0,73
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017","[Pope, Francis, used, his, annual, Christmas, ...",458,261,0,0,38


Turning them into csv files

In [ ]:
# Save results_data_true to a CSV file
results_data_true.to_csv('/drive/My Drive/Colab Notebooks/IST 664/2023/results_data_true.csv', index=False)

# Save results_data_fake to a CSV file
results_data_fake.to_csv('/drive/My Drive/Colab Notebooks/IST 664/2023/results_data_fake.csv', index=False)


#**Interpretations**

### **Top 50 Stop Words - Frequency Analysis of in Real vs. Fake News**

Common Stop Words in Both Real and Fake News Articles:
- We see common stop words like "the","to","a","of" & "and" in both real and fake news articles.


Differences in Stop Word Frequencies:
- There are differences in the frequencies of stop words in real and fake news articles, despite both containing similar words.The stop word "the" is used the most frequently in both, but fake news articles use it more often.

### **Top 50 Content Words - Frequency Analysis**

Common Content Words in Both Real and Fake News Articles:

- Words like "Trump," "wa," and "said," which are likely to appear in news articles regardless of their authenticity, are used in both real and fake news articles.

Differences in Content Word Frequencies:

- Real news articles seem to place more emphasis on words related to politics and the executive branch of government, such as "president," "republican," and "election."

- Contrarily, fake news articles use a wider range of vocabulary, including words with a social or political connotation, such as "like," "Obama," "woman," "think," and "video."

### **Top 50 Bigram Frequencies**

Common Bigrams in Both Real and Fake News Articles:

- "Donald Trump," "White House," "United States," "New York," and "Barack Obama" are common bigrams in both real and fake news articles, reflecting shared topics and figures between the two types of content.

Differences in Bigrams :
- While "Image via" is specific to fake news and not found in real news, "Hillary Clinton" is a prominent bigram in fake news articles, indicating distinctive language patterns. In fake news, "President Obama" appears, but not in actual news. The name "Fox News" is connected to bogus news sources. Exclusive bigrams like "Century Wire," "Fake News," and "Social Media" draw attention to recurrent themes in fake news articles.


### **Top 50 Bigrams by Mutual Information**

Common Bigrams with MI:
- Bigrams with exceptionally high MI scores are found in both real and fake news articles, demonstrating the strong associations between specific phrases or word combinations in each dataset.

- Names and entities that are frequently bigrammed with high MI scores include "abravanel, lesleyabravanel" in fake news and "keefe, bruyette" in legitimate news, indicating that particular people or entities are frequently mentioned or linked to these articles.

- Bigrams with MI scores close to 20, which is a comparatively high value, are present in both datasets, emphasising the strong correlations between specific word pairs in both kinds of news articles.


Differences:

- Both datasets have high MI scores for bigrams, but they differ. Fake news contains unique bigrams like "abravanel, lesleyabravanel," "hemmed, hawed," and "kambree, kawahine," absent in real news, possibly reflecting fake news-specific patterns or entities.

- MI scores in fake news are generally slightly higher, implying the heightened importance of specific word combinations in fake news content.

### **Top 50 Adjectives Analysis**

Common Analysis:
- Adjectives are frequently used to describe and contextualise the content in news articles, both real and fake. Adjectives are essential to language and are used to describe nouns in greater detail.

- The top 50 adjectives in both instances contain terms like "new," "former," "national," "political," "presidential," "military," and "public." These terms are frequently used to describe various facets of events, people, and policies in news articles.


Differences:
- There is a difference between the adjectives used in authentic and fake news articles. Adjectives like "republican," "united," "north," and "international," which indicate a focus on international affairs and politics, are frequently used in real news articles.

- Contrarily, fake news articles place a greater emphasis on words like "hillary," "donald," "liberal," and "illegal," indicating a propensity to use divisive language and politically charged content.



###**FOR TRUE NEWS: Comparison of the results from bigrams by frequencies and by mutual information scores**

Commonalities

- The bigrams primarily reflect words and phrases from the news domain in both the frequency-based and MI-based lists. This includes references to public figures (such as "Barack Obama" and "Donald Trump"), governmental entities (such as "the White House" and "the Supreme Court"), and world events (such as "North Korea" and "Saudi Arabia").

- In both sets of results, proper nouns, such as names of people, places, and organisations, are frequently used. In news articles where particular entities are frequently mentioned, this is to be expected.

Differences:
- The selection criteria make a significant difference between frequency and MI score. The frequency-based list highlights bigrams that are important in general news reporting and are frequently found in the dataset. The MI-based list, in contrast, gives bigrams with high mutual information scores priority. These bigrams may not be widespread, but they are contextually significant within some articles.

- The MI-based list contains bigrams that are contextually specific and may not be as well known as those on the frequency-based list, which tends to capture more general and broadly applicable bigrams. For instance, although "keefe bruyette" and "maale adumim" are uncommon in daily news, they have high MI scores.

- Bigrams based on MI, like "doxycycline hyclate" and "jabha shamiya," are probably related to particular articles or topics in the dataset. Their high MI ratings imply that they offer distinctive context or information in each of their individual articles.



###**FOR FAKE NEWS: Comparison of the results from bigrams by frequencies and by mutual information scores**

- The top 50 bigrams and the top 50 bigrams with Mutual Information (MI) scores in fake news articles have nothing in common. There are no terms that are repeated in either of the two lists of bigrams.
- The bigrams that frequently appear in fake news articles (as indicated by frequency) are likely different from the context-specific and distinctive bigrams identified by MI scores, according to the lack of commonality.

Key Differences:
- The relevance and specificity of the two sets of results are what really set them apart from one another. The frequency-based bigrams identify common themes in fake news articles, which are frequently political in nature. In contrast, MI-based bigrams emphasise less frequent and situation-specific terms that might not always be related to political content.

- It's important to note that neither set of results from the MI-based list is coherent. The variety of terms with high MI scores indicates that fake news articles contain a broad range of context-specific information without following a regular pattern. This is probably due to the wide range of subjects covered and the heterogeneity of fake news sources.



### **Length and Complexity Articles based on Total_Words & Total_Content_Words:**

- Compared to fake news articles, real news articles typically have a higher word count and are more complex. The "Total_Words" column makes this clear, showing that genuine news articles have longer average word counts.

- The "Total_Content_Words" column, which represents the count of content words (aside from stop words), also exemplifies how genuine news articles frequently have more substantial content than their fake counterparts. This suggests that authentic news articles offer more in-depth and educational content.




###**Capitalised Letters(Excluding 'I')**

- Fake news articles appear to use all-capitalized words more frequently when it comes to the use of capitalised words. The "Total_Capitalized_Words" column serves as a hint.
- It's important to keep in mind that this could be a strategic decision made in fake news articles to draw attention to or emphasise certain words.


###**Exclamation Mark(!)**

- The "Total_Exclamation_Marks" column demonstrates how fake news articles frequently use more exclamation points.
- This suggests that sensationalism and excitement may be used in fake news articles to draw readers in.

###**Punctuation Marks**
- The "Total_Punctuation_Marks" column shows that true news articles typically have more punctuation.
- This might be because real news articles use direct quotations, long sentences, and a formal writing style.

###**What I learnt from the analysis?**

- The analysis has shown significant differences between real and fake news articles in terms of length, complexity, and stylistic elements. True news articles typically have longer, more complex sentence structures and more punctuation. Contrarily, exclamation points and capitalised words are frequently used in fake news articles, seemingly in an effort to grab readers' attention. These differences probably correspond to different writing procedures and goals that underlie these two article types.

- Future analyses could focus on the distribution of particular words and phrases that are unique to each category in order to further our understanding of the emotional tone and distinctive features of real and fake news articles. Such investigations would offer insightful understandings into the subtleties and techniques used in these articles.